<a href="https://colab.research.google.com/github/shahriarivari/Persian_sentiment_analysis/blob/main/BERT_notebooks/Train_BERT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#some pip installs

In [ ]:
!pip install tokenizers
!pip install datasets

In [1]:
# some imports
import os
from tokenizers import BertWordPieceTokenizer
from datasets import load_dataset, Dataset

#Loading the dataset
We are using Huggingface's data hub

##Loading the data

In [24]:
# You should just change this part in order to download your
# parts of corpus.
# There are a total of 126 (indexing from 0-> 125) files 0f training text each with size of 1.07 GB roughly
# There are a total of 3 (indexing from 0-> 2) files of testing text each with size of 1.07 GB roughly 
# for more cehck the link -> https://huggingface.co/datasets/SLPL/naab

indices = {
    "train": [81, 14, 3,
              # 94, 35,
              # 41, 28, 67, 55, 79
              ],
    "test": [0,
            #  1
             ]
}

N_FILES = {
    "train": 126,
    "test": 3
}
_BASE_URL = "https://huggingface.co/datasets/SLPL/naab/resolve/main/data/"
data_url = {
    "train": [_BASE_URL + "train-{:05d}-of-{:05d}.txt".format(x, N_FILES["train"]) for x in range(N_FILES["train"])],
    "test": [_BASE_URL + "test-{:05d}-of-{:05d}.txt".format(x, N_FILES["test"]) for x in range(N_FILES["test"])],
}
for index in indices['train']:
    assert index < N_FILES['train']
for index in indices['test']:
    assert index < N_FILES['test']
data_files = {
    "train": [data_url['train'][i] for i in indices['train']],
    "test": [data_url['test'][i] for i in indices['test']]
}


In [ ]:
dataset_train = load_dataset('text', data_files=data_files["train"], use_auth_token=False)
dataset_test = load_dataset('text', data_files=data_files["test"], use_auth_token=False)

##Turn each dataset into a .txt file to feed to the tokenizer

inorder to train the tokenizer we could use the huggingface dataset object or turn our data to .txt file and then feed it to the toknizer. we are using the latter.

In [ ]:
# in this for loop we iteratre through train and tet datset imported from huggingface's hub and write them as .txt file

for index, dataset in enumerate([dataset_train, dataset_test]):
  current_dataset = dataset['train']['text']
  # Save the current slice to a file
  current_file = f'dataset{index + 1}.txt'               # each datset will be named -> "dataset" + index
  print(f"starting writing dataset{index + 1}")
  with open(current_file, 'w', encoding='utf-8') as file:
      for text in current_dataset:
          file.write(text + '\n')

  print(f"finished writing dataset{index + 1}")
  # Free up memory by deleting the variable
  del current_dataset

In [27]:
# Set your paths and file names
tokenizer_output_dir = "bert_tokenizer"
if not os.path.isdir(tokenizer_output_dir):
    os.mkdir(tokenizer_output_dir)

##Training the tokenizer

For more information check the website -> https://huggingface.co/docs/tokenizers/api/trainers#tokenizers.trainers.WordPieceTrainer , https://huggingface.co/docs/tokenizers/api/tokenizer

In [ ]:
# Training a WordPiece Tokenizer
files = ["dataset1.txt", "dataset2.txt"]

# Parameters for Tokenizer Training
vocab_size = 10_000                                                 # The size of the final vocabulary, including all tokens and alphabet.
min_frequency = 20                                                  # The minimum frequency a pair should have in order to be merged.
special_tokens = ["[PAD]", "[MASK]", "[CLS]", "[SEP]", "[UNK]"]     #  A list of special tokens the model should know of.

# Initialize the WordPiece tokenizer for BERT
tokenizer = BertWordPieceTokenizer()
print("starting to train the toeknizer")

# Train the tokenizer
tokenizer.train(
    files=files,
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    show_progress=True,
    special_tokens=special_tokens,
)
print("training is done now saving...")
# Save the trained tokenizer
tokenizer.save_model(tokenizer_output_dir)